In [1]:
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier


/media/Data/PhD/mlaut/venv_mlaut/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/media/Data/PhD/mlaut/venv_mlaut/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/media/Data/PhD/mlaut/venv_mlaut/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/media/Data/PhD/mlaut/venv_mlaut/lib/python3.6/site-

In [2]:
def keras_model_classification(num_classes, input_dim):
    nn_deep_model = OverwrittenSequentialClassifier()
    nn_deep_model.add(Dense(288, input_dim=input_dim, activation='relu'))
    nn_deep_model.add(Dense(144, activation='relu'))
    nn_deep_model.add(Dropout(0.5))
    nn_deep_model.add(Dense(12, activation='relu'))
    nn_deep_model.add(Dense(num_classes, activation='softmax'))

In [9]:
keras_classifier = KerasClassifier(build_fn=keras_model_classification, 
                                num_classes=self.num_classes, 
                                input_dim=self.input_dim,
                                batch_size=100, 
                                epochs=1)

NameError: name 'self' is not defined

In [10]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

base_estimator = {
    'classifier': DecisionTreeClassifier(),
    'regressor': DecisionTreeRegressor()
}

In [11]:
base_estimator['classifier']

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')